In [5]:
import pandas as pd
import matplotlib as plt
from alpha_vantage.timeseries import TimeSeries
import time
import json

In [3]:
#import quandl
#quandl.ApiConfig.api_key = 'VDCe-Sck1jKwogtfPBtN'
#quandl.ApiConfig.verify_ssl = False
#data = quandl.get_table('WIKI', ticker='AAPL')

In [15]:
api_key = '7DOY154L31ZDC2PG'

Stock Call Function

In [87]:
def stock(ticker):
    ticker = ticker.upper()
    ts = TimeSeries(key='7DOY154L31ZDC2PG', output_format='pandas')
    data, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
    df = pd.DataFrame(data)
        
    # Set the index to a column called Date
    df = df.reset_index(level=0)
    df = df.rename(columns={"5. adjusted close": "Adj. Close",
                        "adjHigh": "Adj. High",
                        "adjLow": "Adj. Low",
                        "adjOpen": "Adj. Open",
                        "adjVolume": "Adj. Volume",
                        "4. close": "Close",
                        "date": "Date",
                        "7. dividend amount": "Dividend",
                        "2. high": "High",
                        "3. low": "Low",
                        "1. open": "Open",
                        "8. split coefficient": "Split Ratio",
                        "6. volume": "Volume"})

    # Columns required for prophet
    #df['ds'] = df['Date']
    #df['y'] = df['Adj. Close']
        

    df['Daily Change'] = df['Adj. Close'] - df['Open']
   
    min_date = min(df['Date'])
    max_date = max(df['Date'])
   
    orig_len = df.size
    print('{} Stock Initialized. Data covers {} to {}. Length is {}.'.format(ticker,min_date,max_date, orig_len))

    return df

In [88]:
apple = stock('aapl')

AAPL Stock Initialized. Data covers 1999-11-01 00:00:00 to 2021-11-12 00:00:00. Length is 55460.


In [89]:
apple.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj. Close', 'Volume',
       'Dividend', 'Split Ratio', 'Daily Change'],
      dtype='object')

In [90]:
apple.head()

,Date,Open,High,Low,Close,Adj. Close,Volume,Dividend,Split Ratio,Daily Change
0,2021-11-12,148.43,150.400,147.4800,149.99,149.99,63245197.0,0.0,1.0,1.56
1,2021-11-11,148.96,149.430,147.6810,147.87,147.87,40999950.0,0.0,1.0,-1.09
2,2021-11-10,150.02,150.130,147.8500,147.92,147.92,65187092.0,0.0,1.0,-2.10
3,2021-11-09,150.20,151.428,150.0601,150.81,150.81,56573449.0,0.0,1.0,0.61
4,2021-11-08,151.41,151.570,150.1600,150.44,150.44,55020868.0,0.0,1.0,-0.97


Making Chart Call

In [ ]:
#tesla.plot_stock(stats = ['Daily Change', 'Adj. Volume'],  plot_type='pct')

In [ ]:
    def plot_stock(self, start_date=None, end_date=None, stats=['Adj. Close'], plot_type='basic'):
        
        self.reset_plot()
        
        if start_date is None:
            start_date = self.min_date
        if end_date is None:
            end_date = self.max_date
        
        stock_plot = self.make_df(start_date, end_date)

        colors = ['r', 'b', 'g', 'y', 'c', 'm']
        
        for i, stat in enumerate(stats):
            
            stat_min = min(stock_plot[stat])
            stat_max = max(stock_plot[stat])

            stat_avg = np.mean(stock_plot[stat])
            
            date_stat_min = stock_plot[stock_plot[stat] == stat_min]['Date']
            date_stat_min = date_stat_min[date_stat_min.index[0]]
            date_stat_max = stock_plot[stock_plot[stat] == stat_max]['Date']
            date_stat_max = date_stat_max[date_stat_max.index[0]]
            
            print('Maximum {} = {:.2f} on {}.'.format(stat, stat_max, date_stat_max))
            print('Minimum {} = {:.2f} on {}.'.format(stat, stat_min, date_stat_min))
            print('Current {} = {:.2f} on {}.\n'.format(stat, self.stock.loc[self.stock.index[-1], stat], self.max_date))
            
            # Percentage y-axis
            if plot_type == 'pct':
                # Simple Plot 
                plt.style.use('fivethirtyeight');
                if stat == 'Daily Change':
                    plt.plot(stock_plot['Date'], 100 * stock_plot[stat],
                         color = colors[i], linewidth = 2.4, alpha = 0.9,
                         label = stat)
                else:
                    plt.plot(stock_plot['Date'], 100 * (stock_plot[stat] -  stat_avg) / stat_avg,
                         color = colors[i], linewidth = 2.4, alpha = 0.9,
                         label = stat)

                plt.xlabel('Date'); plt.ylabel('Change Relative to Average (%)'); plt.title('%s Stock History' % self.symbol); 
                plt.legend(prop={'size':10})
                plt.grid(color = 'k', alpha = 0.4); 

            # Stat y-axis
            elif plot_type == 'basic':
                plt.style.use('fivethirtyeight');
                plt.plot(stock_plot['Date'], stock_plot[stat], color = colors[i], linewidth = 3, label = stat, alpha = 0.8)
                plt.xlabel('Date'); plt.ylabel('US $'); plt.title('%s Stock History' % self.symbol); 
                plt.legend(prop={'size':10})
                plt.grid(color = 'k', alpha = 0.4); 
      
        plt.show();

In [ ]:
def plot(start_data=None, end_date=None, stats)